In [14]:
import requests
import pandas as pd
import os
'''from dotenv import load_dotenv


# Laste inn miljøvariabler fra .env-filen
load_dotenv()

# Hent API-nøkkelen fra miljøvariablene
api_key = os.getenv('API_KEY_FROST')'''
API_KEY_FROST = "f9d56ccf-fe79-45ff-970e-959f8e0de1e5"


endpoint = 'https://frost.met.no/observations/v0.jsonld'

from datetime import datetime, timedelta
parameters = {
    'sources': 'SN68090',
    'elements': 'mean(air_temperature P1D)',
    'referencetime': '2010-01-01/2020-01-01',
}


# Issue an HTTP GET request
r = requests.get(endpoint, parameters, auth=("f9d56ccf-fe79-45ff-970e-959f8e0de1e5",""))
# Extract JSON data
json = r.json()

# Check if the request worked, print out any errors
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])


# This will return a Dataframe with all of the observations in a table format
df = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
df = pd.concat([df, row], ignore_index=True)
df = df.reset_index()



Error! Returned status code 412
Message: 412
Reason: No time series found for this combination of parameters, check /observations/availableTimeSeries for more information.


In [10]:
import requests

# Frost API-endepunkt for tilgjengelige tidsserier
endpoint = 'https://frost.met.no/observations/availableTimeSeries/v0.jsonld'

# Parametre for forespørselen
params = {
    'sources': 'SN68SN68090173',  # Stasjon-ID
    'referencetime': '2020-01-01/2021-01-01',  # Tidsperiode
}

# Utfør HTTP GET-forespørsel
r = requests.get(endpoint, params=params, auth=("f9d56ccf-fe79-45ff-970e-959f8e0de1e5", ""))

# Sjekk om forespørselen var vellykket og hent JSON-data
if r.status_code == 200:
    json = r.json()
    if 'data' in json:
        print("Tilgjengelige tidsserier for stasjon SN68173:")
        for item in json['data']:
            print(f"Element: {item['elementId']}, Referencetime: {item['referencetime']}")
    else:
        print("Ingen tidsserier funnet for denne stasjonen og tidsperioden.")
else:
    print(f"Error! Returned status code {r.status_code}")


Error! Returned status code 400


In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

# Laste inn miljøvariabler fra .env-filen
load_dotenv()

# Hent API-nøkkelen fra miljøvariablene
api_key = os.getenv('API_KEY_FROST')

# Endpoint for API-kallet (antar at du har en endpoint-URL som mangler)
endpoint = 'https://frost.met.no/observations/v0.jsonld'  # Eksempel på API-endepunkt

# Parametre for forespørselen
parameters = {
    'sources': 'SN90450',  # Kilde-ID
    'elements': 'max(snow_coverage_type P3M)',  # Eksempel på element (juster etter behov)
    'referencetime': '2020-01-01/2021-01-01',  # Tidsperiode
}

# Utfør HTTP GET-forespørsel
r = requests.get(endpoint, params=parameters, auth=("f9d56ccf-fe79-45ff-970e-959f8e0de1e5", ""))

# Sjekk om forespørselen var vellykket og hent JSON-data
if r.status_code == 200:
    json = r.json()
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print(f'Error! Returned status code {r.status_code}')
    print(f'Message: {json["error"]["message"]}')
    print(f'Reason: {json["error"]["reason"]}')

# Denne delen bearbeider JSON-dataene til en Pandas DataFrame
df = pd.DataFrame()  # Tom DataFrame for å samle data
for i in range(len(data)):
    # Ekstrakt observations for hver datarow
    row = pd.DataFrame(data[i]['observations'])
    
    # Legg til tid og kilde-ID
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    
    # Slå sammen med hoved DataFrame
    df = pd.concat([df, row], ignore_index=True)

# Resett indeksen etter sammenlegging
df = df.reset_index(drop=True)

# Sjekk resultatet
print(df.head())  # Skriver ut de første radene for å verifisere data

# Hente parametrene for Frost API
parameters = get_frost_parameters('SN90450', 'max(snow_coverage_type P3M)', '2020-01-01', '2021-01-01')

# Endpoint for API-kallet (pass på at endpointet er riktig)
endpoint = 'https://frost.met.no/observations/v0.jsonld'

# Utfør HTTP GET-forespørsel med de hentede parameterne
r = requests.get(endpoint, params=parameters, auth=("f9d56ccf-fe79-45ff-970e-959f8e0de1e5", ""))

# Sjekk om forespørselen var vellykket og hent JSON-data
if r.status_code == 200:
    json = r.json()
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print(f'Error! Returned status code {r.status_code}')
    print(f'Message: {json["error"]["message"]}')
    print(f'Reason: {json["error"]["reason"]}')



In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

# Laste inn miljøvariabler fra .env-filen
load_dotenv()

# Hent API-nøkkelen fra miljøvariablene
api_key = os.getenv('API_KEY_FROST')

# Funksjon for å hente parametere for Frost API
def get_frost_parameters(source_id, elements, start_date, end_date):
    """
    Funksjon for å hente parametere som skal brukes til å gjøre en forespørsel til Frost API.
    
    Arguments:
    - source_id: Kilde-ID (for eksempel: 'SN90450')
    - elements: Dataelementer som ønskes (for eksempel: 'max(snow_coverage_type P3M)')
    - start_date: Startdato i formatet 'YYYY-MM-DD' (for eksempel: '2020-01-01')
    - end_date: Sluttdato i formatet 'YYYY-MM-DD' (for eksempel: '2021-01-01')
    
    Return:
    - params: En dictionary med parametrene som skal sendes til API-et.
    """
    parameters = {
        'sources': source_id,  # Kilde-ID som input
        'elements': elements,  # Dataelementer som input
        'referencetime': f'{start_date}/{end_date}',  # Tidsintervall som input
    }
    
    return parameters

# Hente parametrene for Frost API
parameters = get_frost_parameters('SN90450', 'max(snow_coverage_type P3M)', '2020-01-01', '2021-01-01')

# Endpoint for API-kallet (pass på at endpointet er riktig)
endpoint = 'https://frost.met.no/observations/v0.jsonld'

# Utfør HTTP GET-forespørsel med de hentede parameterne
r = requests.get(endpoint, params=parameters, auth=("f9d56ccf-fe79-45ff-970e-959f8e0de1e5", ""))

# Sjekk om forespørselen var vellykket og hent JSON-data
if r.status_code == 200:
    json = r.json()
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print(f'Error! Returned status code {r.status_code}')
    print(f'Message: {json["error"]["message"]}')
    print(f'Reason: {json["error"]["reason"]}')

# Bearbeide JSON-dataene til en Pandas DataFrame
df = pd.DataFrame()  # Tom DataFrame for å samle data
for i in range(len(data)):
    # Ekstrakt observations for hver datarow
    row = pd.DataFrame(data[i]['observations'])
    
    # Legg til tid og kilde-ID
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    
    # Slå sammen med hoved DataFrame
    df = pd.concat([df, row], ignore_index=True)

# Resett indeksen etter sammenlegging
df = df.reset_index(drop=True)

# Sjekk resultatet
print(df.head())  # Skriver ut de første radene for å verifisere data
